In [1]:
import numpy as np

import requests
from bs4 import BeautifulSoup
import pickle

from pymongo import MongoClient
import time

import time
import re

import os

In [2]:
os.chdir('Data')

In [3]:
def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [3]:
def get_job_info(soup):
    
    try:
        job_title = soup.find(name='h3', attrs={'class':'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'}).text
    except:
        job_title = ''
        
    try:
        company_name = soup.find(name='div', attrs={'class':'icl-u-lg-mr--sm icl-u-xs-mr--xs'}).text
    except:
        company_name = ''
        
    try:
        salary = soup.find(name='span', attrs={'class':'icl-u-xs-mr--xs'}).text
    except:
        salary = ''
    
    try:
        job_description_full = soup.find(name='div', attrs={'class':'jobsearch-JobComponent-description icl-u-xs-mt--md'})
        
        try:
            job_description_full = soup.find(name='div', attrs={'class':'jobsearch-JobComponent-description icl-u-xs-mt--md'})
            listed_items = []
            for list_item in job_description_full.find_all(name='li'):
                listed_items.append(list_item.text)
            # Adds '. ' because most list items are missing periods
            listed_items = '. '.join(listed_items)
            listed_items = re.sub('\n', ' ', listed_items)
            # Remove double space
            listed_items = re.sub('  ', ' ', listed_items)
            # Remove double period if list items had periods
            listed_items = re.sub('\.\.', '.', listed_items)
        except:
            listed_items = ''
            
        # Paragraph Items
            
        try:
            paragraph_items = []
            for paragraph_item in job_description_full.find_all(name='p'):
                paragraph_items.append(paragraph_item.text)

            # removes things like: 'Specifically, the Lisp engineer will:' that prepend listed_items
            paragraph_items = [paragraph_item for paragraph_item in paragraph_items if len(paragraph_item) > 50]
            # Puts all paragraph items into a single string
            paragraph_items = ''.join(paragraph_items)
            paragraph_items = re.sub('\n', ' ', paragraph_items)
            
            if len(paragraph_items) < 1:
                try:
                    job_description_full = soup.find(name='div', attrs={'class':'jobsearch-JobComponent-description icl-u-xs-mt--md'})
                    paragraph_items = []
                    for paragraph_item in job_description_full.find_all(name='div'):
                        paragraph_items.append(paragraph_item.text)

                    # removes things like: 'Specifically, the Lisp engineer will:'
                    paragraph_items = [paragraph_item for paragraph_item in paragraph_items if len(paragraph_item) > 50]
                    # Puts all paragraph items into a single string
                    paragraph_items = ''.join(paragraph_items)
                    paragraph_items = re.sub('\n', ' ', paragraph_items)
                except: 
                    paragraph_items = ''
   
        except:
            paragraph_items = ''
    
    except:
        listed_items, paragraph_items = ['','']
            
    return job_title, company_name, salary, listed_items, paragraph_items

### Connecting to MongoDB

In [4]:
client = MongoClient()

In [5]:
db = client.jobs_data

In [16]:
db.list_collection_names()

['Chicago_Jobs',
 'Los_Angeles_Jobs',
 'SF_Bay_Area_Jobs',
 'Seattle_Jobs',
 'New_York_Jobs']

In [33]:
pickles = ['individual_listings__chicago',
           'individual_listings__LosAngeles',
#            'individual_listings__NewYork',
#            'individual_listings__SanFranciscoBayArea',
#            'individual_listings__Seattle'
          ]
collections = ['Chicago_Jobs',
               'Los_Angeles_Jobs',
#                'New_York_Jobs',
#                'SF_Bay_Area_Jobs',
#                'Seattle_Jobs'
              ]

In [36]:
def scrape_to_mongo(pickles, collections):
    
    i = 0
    
    for pickl, collection in zip(pickles, collections):
        pickle_in = open(pickl,'rb')
        listings_urls = pickle.load(pickle_in)
        pickle_in.close()

        listings_urls = [item for sublist in listings_urls for item in sublist]
        listings_urls = list(set(listings_urls))

        for url in listings_urls:

            new_item = {}

            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')
            job_title, company_name, salary, listed_items, paragraph_items = get_job_info(soup)
            time.sleep(np.random.poisson(100)/50)

            new_item['url'] = url
            new_item['job_title'] = job_title 
            new_item['company_name'] = company_name
            new_item['salary'] = salary
            new_item['listed_items'] = listed_items
            new_item['paragraph_items'] = paragraph_items

            time.sleep(np.random.poisson(100)/50)

            eval('db.'+collection+'.insert_one(new_item)')

            if i%10 == 0:
                print(i)

            i += 1
            
            print(new_item)

            del new_item
            del soup
            del job_title
            del company_name
            del salary
            del listed_items
            del paragraph_items
        del listings_urls

In [38]:
db.Chicago_Jobs.insert_one({'test': 'test2'})

In [22]:
db.Chicago_Jobs.insert_one({'test': 'test'})

In [37]:
scrape_to_mongo(pickles, collections)

0
{'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D2gG673WNVYM892ipXKrjHhau7wRi_VpqZd1WRJWp7_2qHY2UFfJsPgB-MwPwnX2J9FHgaNSwvFqIxt7eAz_WDuYxFy3PjUi6rZN8-RFoKGa4_RFbgOW3d1AJwMGARdsx1qmc9QmoQNA8VbFrjbiSFzxFzXA9mzOzc6KBAPPVOw4GUOfrAqJXmd4mq2X2plUZQGbUCfpwnix-Vz611qhqOIfAie3DLbxNx9QRwQbTYTllVbkie-2IB1e7igf8rtrYsKYI9c3blEsUhq66t1qoooWdp4nNVaMIvRMj51Fgs0XEnTr1d1GTJKS6XJfA5EOXLJxQ_2qFOx3XJA7WrlcqRuSEBNfVD_04M3JySSzeZ7CM0dgcxxE7n2hmB_dMxl7-ZDfMsgmM8jkB7k2b6q0Qrisa5nxJw29Or5jD9znfPHWas-Wjd_Q3Q_vr-bFAxNZ8p6KFkrPtKpA==&p=12&fvj=1&vjs=3', 'job_title': 'Ecommerce Merchandising Analyst', 'company_name': 'U.S Auto Parts Network, Inc', 'salary': '', 'listed_items': "Conceive, design, and develop product positioning and messaging for new and current products that differentiate our product and our brands in terms of price, quality, target market, and sales channel. Effectively merchandise products within the product category in an effort to influence conversion rate and sales, while balancin

KeyboardInterrupt: 

In [19]:
db.Chicago_Jobs.find()

In [ ]:
pickles = ['individual_listings__chicago',
           'individual_listings__LosAngeles',
#            'individual_listings__NewYork',
#            'individual_listings__SanFranciscoBayArea',
#            'individual_listings__Seattle'
          ]
collections = ['Chicago_Jobs',
               'Los_Angeles_Jobs',
#                'New_York_Jobs',
#                'SF_Bay_Area_Jobs',
#                'Seattle_Jobs'
              ]

In [47]:
i = 0

pickle_in = open('individual_listings__LosAngeles', 'rb')
listings_urls = pickle.load(pickle_in)
pickle_in.close()

listings_urls = [item for sublist in listings_urls for item in sublist]
listings_urls = list(set(listings_urls))

for url in listings_urls:

    new_item = {}

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    job_title, company_name, salary, listed_items, paragraph_items = get_job_info(soup)
    time.sleep(np.random.poisson(100)/50)

    new_item['url'] = url
    new_item['job_title'] = job_title 
    new_item['company_name'] = company_name
    new_item['salary'] = salary
    new_item['listed_items'] = listed_items
    new_item['paragraph_items'] = paragraph_items

    time.sleep(np.random.poisson(100)/50)

    db.Los_Angeles_Jobs.insert_one(new_item)

    if i%10 == 0:
        print(i)

    i += 1

    print(new_item)

    del new_item
    del soup
    del job_title
    del company_name
    del salary
    del listed_items
    del paragraph_items

0
{'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D2gG673WNVYM892ipXKrjHhau7wRi_VpqZd1WRJWp7_2qHY2UFfJsPgB-MwPwnX2J9FHgaNSwvFqIxt7eAz_WDuYxFy3PjUi6rZN8-RFoKGa4_RFbgOW3d1AJwMGARdsx1qmc9QmoQNA8VbFrjbiSFzxFzXA9mzOzc6KBAPPVOw4GUOfrAqJXmd4mq2X2plUZQGbUCfpwnix-Vz611qhqOIfAie3DLbxNx9QRwQbTYTllVbkie-2IB1e7igf8rtrYsKYI9c3blEsUhq66t1qoooWdp4nNVaMIvRMj51Fgs0XEnTr1d1GTJKS6XJfA5EOXLJxQ_2qFOx3XJA7WrlcqRuSEBNfVD_04M3JySSzeZ7CM0dgcxxE7n2hmB_dMxl7-ZDfMsgmM8jkB7k2b6q0Qrisa5nxJw29Or5jD9znfPHWas-Wjd_Q3Q_vr-bFAxNZ8p6KFkrPtKpA==&p=12&fvj=1&vjs=3', 'job_title': 'Ecommerce Merchandising Analyst', 'company_name': 'U.S Auto Parts Network, Inc', 'salary': '', 'listed_items': "Conceive, design, and develop product positioning and messaging for new and current products that differentiate our product and our brands in terms of price, quality, target market, and sales channel. Effectively merchandise products within the product category in an effort to influence conversion rate and sales, while balancin

KeyboardInterrupt: 

In [45]:
pickle_in = open('individual_listings__LosAngeles', 'rb')
listings_urls = pickle.load(pickle_in)
pickle_in.close()